# TEP detailed analysis

In [ ]:
%matplotlib inline
%autosave 5

In [ ]:
# necessary modules -------------------------------------------------------------------------------------
import pyfesom2 as pf
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.patches as mpatches

from netCDF4 import Dataset

import time
from datetime import date

#import sys
#sys.path.append('home/ollie/mozeisin/SkillMetrics')

import cartopy.crs as ccrs
import cartopy.feature as cfeature
#import pangaeapy.pandataset as pd

import warnings
warnings.simplefilter("ignore")

# own py_recom modules ----------------------------------------------------------------------
#!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

#from load_interp_WOA_python3 import WOAdata
from plot_Taylor_normalized import plt_Taylor_norm

In [ ]:
# After changing these parameters here, save the notebook and clear the output. 
# Then run this cell and the cells with the desired parameters.

# run specification -------------------------------------------------------------------------------------
runid      =  'fesom'
layerwise    = False
resultpath = '/work/ollie/mozeisin/results/f2r1.2/tep1' 
meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

# period of analysis 
first_year = 1950
last_year  = 1980
    
years = np.arange(first_year,last_year+1,1)
years_last10 = np.arange(last_year-10,last_year+1,1)

# specification of analysis ------------------------------------------------------------------------------------
depth = 0

layerwise = False
mapproj = 'pc'
#Map projection. Options are Mercator (merc), Plate Carree (pc),
#North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)
#Robinson projection is quite time consuming.

# export of analysis ------------------------------------------------------------------------------------
today = date.today().strftime("_%Y_%m_%d")

savepath = '/home/ollie/mozeisin/evaluation/mo_files/tep1/'
savefig = True
htmlname     =  'tep1' + today +'.html'
htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'
    

In [ ]:
# just to check
print('Processing years {4} to {5}\n\nReading out of {0}\nStoring graphs to {1}\nStoring html to {2} as {3}'.format(
    resultpath, savepath, htmlpath, htmlname,years[0],years[-1]))

print('\nLast ten years are {0}'.format(years_last10))

# Load mesh

In [ ]:
# mesh initialization -----------------------------------------------------------
mesh = pf.load_mesh(meshpath)

lon_fesom = mesh.x2
lat_fesom = mesh.y2

#print(mesh.zlev)

# grid for intrpolation to 1x1 -----------------------------------------------------------
lat         = np.arange(-89.5,90.,1.)
lon         = np.arange(-179.5,180.,1.)
latdic, londic = np.meshgrid(lat, lon)

# Define regions

In [ ]:
## Arctic Oean, N of 70 -------------------------------------------------------------------------------
box_AO =[-180, 180, 70, 90] 
left, right, down, up = box_AO
mask_AO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

mask_AO_1x1_lon = ((lon >= left) & (lon <= right))
mask_AO_1x1_lat = ((lat >= down) & (lat <= up))

#nod_area_AO = nod_area[:,mask_AO]
#nod_area_AO_surface = nod_area[0,mask_AO]

## Fram Strait -------------------------------------------------------------------------------
box_FRAM =[-20, 10, 77, 82]
left, right, down, up = box_FRAM
mask_FRAM = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

## Souther Oean -------------------------------------------------------------------------------
box_SO =[-180, 180, -90, -50] 
left, right, down, up = box_SO
mask_SO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

#nod_area_SO = nod_area[:,mask_SO]
#nod_area_SO_surface = nod_area[0,mask_SO]
#print(np.shape(nod_area_SO_surface))
        

# Functions

In [ ]:
def mygrid(m,grid=False):
    '''
    Draw cartopy natural earth feature to axis m
    '''
    #m.coastlines(resolution='110m', color='black', linewidth=1)
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', '110m',edgecolor='k',facecolor=cfeature.COLORS['land'])
    m.add_feature(land_poly)

    if grid:
        g1 = m.gridlines(draw_labels = True)
        g1.xlabels_top = False
        g1.ylabels_right = False
        g1.xlabel_style = {'size': 16, 'color': 'gray'}
        g1.ylabel_style = {'size': 16, 'color': 'gray'}

# Chlorophyll

In [ ]:
#!jupyter nbconvert --to=python Chlsurf_OCCCI_Comparison_python3.ipynb
from Chlsurf_OCCCI_Comparison_python3 import Chlsurf_OCCCI_comp

# OCCCI   
matfileChlsurf           = '/work/ollie/projects/MarESys/evaluation/climatology_annual_chl_1deg_OCCCI_2012_2015.mat'

# analyze only last 10 years, 1 uppermost depth layer
Chl = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,
                          last_year-10,last_year,
                          mapproj=mapproj,savefig=savefig)


In [ ]:
# analyze only last 10 years, 2 uppermost depth layers
Chl_n2 = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,
                          last_year-10,last_year,
                         n_levels=2,
                          mapproj=mapproj,savefig=savefig)



# TEP

In [ ]:
NCfesom = resultpath + '/TEP.fesom.1948.nc'
!ncdump -h $NCfesom

In [ ]:
# various depths

units = 'TEP-C [mmol m$^{-3}$]'

depthi = 0
TEP_0 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_0='TEP {0}-{1} at 0-5 m'.format(years_last10[0],years_last10[-1])

depthi = 5
TEP_1 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_1='TEP {0}-{1} at 5-10 m'.format(years_last10[0],years_last10[-1])

depthi = 10
TEP_2 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_2='TEP {0}-{1} at 10-20 m'.format(years_last10[0],years_last10[-1])

depthi = 20
TEP_3 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_3='TEP {0}-{1} at 20-30 m'.format(years_last10[0],years_last10[-1])

In [ ]:
fig = pf.plot(mesh, [TEP_0,TEP_1,TEP_2,TEP_3], mapproj =  mapproj, units=units, 
              rowscol = (2,2), figsize = (15,10),
              levels = (0,15,61), #cmap = 'viridis',
              titles=[label_0,label_1,label_2,label_3])

if savefig:
    plt.savefig(savepath+'TEP_distribution_multidepths.png',dpi=300)

# integrals

In [ ]:
# load nodal area -------------------------------------------------------------------------------------
f = Dataset(resultpath + '/fesom.mesh.diag.nc', 'r')
nod_area = f.variables['nod_area'][:] # Moritz
nod_area_surface = nod_area[0,:]

#print('shape nod_area: {0}\nshape nod_area_surface: {1}'.format(np.shape(nod_area),np.shape(nod_area_surface)))
        
# calculating mean TEP per year -------------------------------------------------------------------------------
TEP00glob_total = np.zeros(len(years))
TEP05glob_total  = np.zeros(len(years))
TEP10glob_total = np.zeros(len(years))
TEP20glob_total   = np.zeros(len(years))

TEP00ao_total = np.zeros(len(years))
TEP05ao_total = np.zeros(len(years))
TEP10ao_total = np.zeros(len(years))
TEP20ao_total = np.zeros(len(years))

TEP00FRAM_total = np.zeros(len(years))
TEP05FRAM_total = np.zeros(len(years))
TEP10FRAM_total = np.zeros(len(years))
TEP20FRAM_total = np.zeros(len(years))

TEP00so_total = np.zeros(len(years))
TEP05so_total = np.zeros(len(years))
TEP10so_total = np.zeros(len(years))
TEP20so_total = np.zeros(len(years))


In [ ]:
for i in range(len(years)):
    ind_year = int(years[i])
    #print('loading {0}'.format(ind_year))
    # TEP:units = "mmolC/m3"
    
    # 0m ############################
    TEP00glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 0,
                        compute=True, runid='fesom', silent=True)
    TEP00glob_total[i] = np.nanmean(TEP00glob)#*nod_area_surface)
    TEP00ao_total[i] = np.nanmean(TEP00glob[mask_AO])
    #TEP00ao_total[i] = np.nanmean(TEP00ao)
    TEP00FRAM_total[i] = np.nanmean(TEP00glob[mask_FRAM])
    TEP00so_total[i] = np.nanmean(TEP00glob[mask_SO])
    #TEP00so_total[i] = np.nanmean(TEP00so)
    
    # 5 m ############################
    TEP05glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 5,
                        compute=True, runid='fesom', silent=True)

    TEP05glob_total[i] = np.nanmean(TEP05glob)
    TEP05ao_total[i] = np.nanmean(TEP05glob[mask_AO])
    TEP05FRAM_total[i] = np.nanmean(TEP05glob[mask_FRAM])
    TEP05so_total[i] = np.nanmean(TEP05glob[mask_SO])

    # 10 m ############################
    TEP10glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 10,
                        compute=True, runid='fesom', silent=True)

    TEP10glob_total[i] = np.nanmean(TEP10glob)
    TEP10ao_total[i] = np.nanmean(TEP10glob[mask_AO])
    TEP10FRAM_total[i] = np.nanmean(TEP10glob[mask_FRAM])
    TEP10so_total[i] = np.nanmean(TEP10glob[mask_SO])
    
    # 20 m ############################
    TEP20glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 20,
                        compute=True, runid='fesom', silent=True)

    TEP20glob_total[i] = np.nanmean(TEP20glob)
    TEP20ao_total[i] = np.nanmean(TEP20glob[mask_AO])
    TEP20FRAM_total[i] = np.nanmean(TEP20glob[mask_FRAM])
    TEP20so_total[i] = np.nanmean(TEP20glob[mask_SO])

In [ ]:
# plotting total TEP -------------------------------------------------------------------------------        
fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (0,7)

# 0 m ############################

plt.subplot(2, 2, 1)
plt.plot(years,TEP00glob_total,'.-',label='0-5 m')
plt.plot(years,TEP05glob_total,'.-',label='5-10 m')
plt.plot(years,TEP10glob_total,'.-',label='10-20 m')
plt.plot(years,TEP20glob_total,'.-',label='20-30 m')
plt.title('Global mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 2)
plt.plot(years,TEP00ao_total,'.-',label='0-5 m')
plt.plot(years,TEP05ao_total,'.-',label='5-10 m')
plt.plot(years,TEP10ao_total,'.-',label='10-20 m')
plt.plot(years,TEP20ao_total,'.-',label='20-30 m')
plt.title('N of 70N mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 3)
plt.plot(years,TEP00so_total,'.-',label='0-5 m')
plt.plot(years,TEP05so_total,'.-',label='5-10 m')
plt.plot(years,TEP10so_total,'.-',label='10-20 m')
plt.plot(years,TEP20so_total,'.-',label='20-30 m')
plt.title('S. Ocean mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 4)
plt.plot(years,TEP00FRAM_total,'.-',label='0-5 m')
plt.plot(years,TEP05FRAM_total,'.-',label='5-10 m')
plt.plot(years,TEP10FRAM_total,'.-',label='10-20 m')
plt.plot(years,TEP20FRAM_total,'.-',label='20-30 m')
plt.title('Fram Strait mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

if savefig:
    plt.savefig(savepath+'TEP_timeseries.png',dpi=300)

# Interpolation to 1x1

In [ ]:
TEP_0_interp = pf.fesom2regular(data = TEP_0,mesh = mesh,lons = londic,lats = latdic)
TEP_1_interp = pf.fesom2regular(data = TEP_1,mesh = mesh,lons = londic,lats = latdic)
TEP_2_interp = pf.fesom2regular(data = TEP_2,mesh = mesh,lons = londic,lats = latdic)
TEP_3_interp = pf.fesom2regular(data = TEP_3,mesh = mesh,lons = londic,lats = latdic)

In [ ]:
if True: # interpolation check            
    print('\nFESOM interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
        np.nanmin(TEP_0),np.nanmax(TEP_0),np.nanmean(TEP_0),
        np.nanmin(TEP_0_interp),np.nanmax(TEP_0_interp),np.nanmean(TEP_0_interp)))
     

# North Polar Stereo

In [ ]:
# North Polar Stereo Projection
if False:
    fig = plt.figure(figsize=(8,8))
    ax1 = plt.subplot(projection = ccrs.NorthPolarStereo())

    m1 = ax1.pcolormesh(londic, latdic, TEP_0_interp, 
        transform = ccrs.PlateCarree(),
        #norm=colors.LogNorm(vmin=np.nanmin(TEP_0_interp),vmax=np.nanmax(TEP_0_interp)),
        cmap='viridis')

    #ax1.coastlines(resolution='110m', color='black', linewidth=1)
    mygrid(ax1,grid=True)

    plot_extent = [-180, 180, 70, 90]
    ax1.set_extent(plot_extent, crs=ccrs.PlateCarree())

    cbar = fig.colorbar(m1,orientation = 'horizontal',
                        fraction=0.1, pad=0.05, shrink = 0.8) 
    cbar.set_label('[mmol C m$^{-3}$]\n'+label_0, fontsize=20,)

    #plt.title('Interpolated '+label_0)

    if savefig:
        plt.savefig(savepath+'TEP_interp_NPS.png', dpi=300)

In [ ]:
# North Polar Stereo Projection
fig = plt.figure(figsize=(20,15))
plot_extent = [-180, 180, 70, 90]

# 0 m
ax1 = plt.subplot(1,3,1,projection = ccrs.NorthPolarStereo())
m1 = ax1.pcolormesh(londic, latdic, TEP_0_interp, 
    transform = ccrs.PlateCarree(),
    #norm=colors.LogNorm(vmin=np.nanmin(TEP_0_interp),vmax=np.nanmax(TEP_0_interp)),
    cmap='viridis')
#ax1.coastlines(resolution='110m', color='black', linewidth=1)
mygrid(ax1,grid=True)
ax1.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax1.set_title('Interpolated '+label_0)

# 5 m
ax2 = plt.subplot(1,3,2,projection = ccrs.NorthPolarStereo())
m2 = ax2.pcolormesh(londic, latdic, TEP_1_interp, 
    transform = ccrs.PlateCarree(),
    cmap='viridis')
mygrid(ax2,grid=True)
ax2.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax2.set_title('Interpolated '+label_1)

# 10 m
ax3 = plt.subplot(1,3,3,projection = ccrs.NorthPolarStereo())
m3 = ax3.pcolormesh(londic, latdic, TEP_2_interp, 
    transform = ccrs.PlateCarree(),
    cmap='viridis')
mygrid(ax3,grid=True)
ax3.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax3.set_title('Interpolated '+label_2)

# colorbar
cbar = fig.colorbar(m1,ax=(ax1,ax2,ax3),orientation = 'horizontal',
                    fraction=0.1, pad=0.05, shrink = 0.5) 
cbar.set_label('TEP-C [mmol C m$^{-3}$]', fontsize=20,)

if savefig:
    plt.savefig(savepath+'TEP_interp_multidepths_NPS.png', dpi=300)

# TEP in relation to CHl.a
figure y= log TEP XG eq , x= log Chla cmop. Zamanillo 2019 and Yamada 2015

### Convert TEP-C to TEP$_{color}$
TEP-C [$\mu$mol L$^{-1}$] = 0.05 TEP$_{color}$ [mg Xeq. L$^{-1}$]

In [ ]:
TEP_0_interp_mugXeqL = TEP_0_interp * 20 * 1000
TEP_1_interp_mugXeqL = TEP_1_interp * 20 * 1000

### extracting regions

In [ ]:
## Arctic Oean, N of 70 
box_AO=[-180, 180, 70, 90] 
left, right, down, up = box_AO
mask_AO_1x1_lon = ((lon >= left) & (lon <= right))
mask_AO_1x1_lat = ((lat >= down) & (lat <= up))


if False: # control of cuting region
    fig = plt.figure(figsize=(10,10))
    ax1 = plt.subplot(projection = ccrs.NorthPolarStereo())

    m1 = plt.pcolormesh(londic[mask_AO_1x1_lon][:,mask_AO_1x1_lat], 
                        latdic[mask_AO_1x1_lon][:,mask_AO_1x1_lat], 
                        TEP_0_interp_mugXeqL[mask_AO_1x1_lon][:,mask_AO_1x1_lat], 
        transform = ccrs.PlateCarree(),
        cmap='viridis')

    #ax1.coastlines(resolution='110m', color='black', linewidth=1)
    mygrid(ax1,grid=True)

    cbar = fig.colorbar(m1,#orientation = 'horizontal',
                        fraction=0.1, pad=0.1,shrink = 0.8) 
    cbar.set_label('TEP-C [mg XG eq L$^{-3}$]', fontsize=20)

    plt.title('Interpolated '+label_0)

### set Chl.a minimum to 0.01 mg/m3

In [ ]:
def set_Chl_min(value,Chl_interp,TEP_interp):
    
    ind = np.where(Chl_interp > value)
    Chl_interp_min = Chl_total_interp_AO[ind]
    print('Number of Chl.a datapoints omitted: {0}'.format(len(Chl_interp) - len(Chl_interp_min) ))

    # apply to TEP as well
    TEP_interp_min = TEP_interp[ind]
    
    return [Chl_interp_min,TEP_interp_min]


In [ ]:
# 1 layer data ######################
# Chl
Chl_total_interp_AO = Chl.Chl_total_interp[:,mask_AO_1x1_lat].ravel()

# TEP
TEP_0_interp_AO_mugXeqL = TEP_0_interp_mugXeqL[:,mask_AO_1x1_lat].ravel()

# 2 layer data ######################
# Chl
Chl_n2_total_interp_AO = Chl_n2.Chl_total_interp[:,mask_AO_1x1_lat].ravel()

# TEP
TEP_n2_mean = (TEP_0_interp_mugXeqL + TEP_1_interp_mugXeqL)/2
TEP_n2_interp_AO_mugXeqL = TEP_n2_mean[:,mask_AO_1x1_lat].ravel()

# plotting relation ######################

#x1 = np.log10(Chl_total_interp_AO_min)
#y2 = np.log10(TEP_0_interp_AO_min_mugXeqL)

x1, y1 = np.log10(set_Chl_min(0.01,Chl_total_interp_AO,TEP_0_interp_AO_mugXeqL))

x2, y2 = np.log10(set_Chl_min(0.05,Chl_total_interp_AO,TEP_0_interp_AO_mugXeqL))

# 2 layers ######################
x1_n2, y1_n2 = np.log10(set_Chl_min(0.01,Chl_n2_total_interp_AO,TEP_n2_interp_AO_mugXeqL))
x2_n2, y2_n2 = np.log10(set_Chl_min(0.05,Chl_n2_total_interp_AO,TEP_n2_interp_AO_mugXeqL))

In [ ]:
# linear regression
def get_res(x,y):
    res = stats.linregress(x,y)
    log = 'Regression: \nslope = %.2f intercept = %.2f, std error = %.3f\nR-squared = %.5f' % (
        res.slope, res.intercept, res.stderr, res.rvalue**2)
    print(log)
    return res, log

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.plot(x1,y1,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.01 mg m$^{-3}$')
plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x1,y1)
plt.plot(x1, res.intercept + res.slope*x1, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.subplot(2,2,2)
plt.plot(x2,y2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.1 mg m$^{-3}$')
#plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x2,y2)
plt.plot(x2, res.intercept + res.slope*x2, 'r', label='fitted line')
plt.text(-2,0.5,log)

# 2 layer data ################
plt.subplot(2,2,3)
plt.plot(x1_n2,y1_n2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.01 mg m$^{-3}$\nn_layer = 2')
plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x1_n2,y1_n2)
plt.plot(x1, res.intercept + res.slope*x1, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.subplot(2,2,4)
plt.plot(x2_n2,y2_n2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.1 mg m$^{-3}$\nn_layer = 2')
#plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x2_n2,y2_n2)
plt.plot(x2, res.intercept + res.slope*x2, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.tight_layout()
if savefig:
    plt.savefig(savepath+'Chl_TEP_regression.png', dpi=300)


# Fram Strait Chl.a - TEP relation

### Extracting regions

In [ ]:
## Fram Strait 
# box_FRAM =[-20, 10, 77, 82] # see above
left, right, down, up = box_FRAM
mask_FRAM_1x1_lon = ((lon >= left) & (lon <= right))
mask_FRAM_1x1_lat = ((lat >= down) & (lat <= up))

if False:
    print(np.shape(mask_FRAM_1x1_lat))
    print(np.shape(mask_FRAM_1x1_lon))
    print(np.shape(TEP_0_interp))

if True: # control of cuting region
    fig = plt.figure(figsize=(20,10))
    
    ax1 = plt.subplot(1,2,1,projection = ccrs.NorthPolarStereo())
    m1 = plt.pcolormesh(londic[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat], 
                        latdic[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat], 
                        TEP_0_interp_mugXeqL[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat], 
        transform = ccrs.PlateCarree(),
        cmap='viridis')
    mygrid(ax1,grid=True)
    cbar = fig.colorbar(m1,
                        orientation = 'horizontal',
                        fraction=0.1, pad=0.1,shrink = 0.8) 
    cbar.set_label('TEP-C [mg XG eq L$^{-3}$]', fontsize=20)
    plt.title('Interpolated '+label_0)
    
    # chlorophyll
    ax2 = plt.subplot(1,2,2,projection = ccrs.NorthPolarStereo())
    m2 = plt.pcolormesh(londic[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat], 
                        latdic[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat], 
                        Chl.Chl_total_interp[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat], 
        transform = ccrs.PlateCarree(),
        cmap='viridis')
    mygrid(ax2,grid=True)
    cbar = fig.colorbar(m2,
                        orientation = 'horizontal',
                        fraction=0.1, pad=0.1,shrink = 0.8) 
    cbar.set_label('Chl.a [mg L$^{-3}$]', fontsize=20)

    plt.title('Interpolated Chl.a')

In [ ]:
# 1 layer data ######################
# Chl
#Chl_total_interp_FRAM = Chl.Chl_total_interp[:,mask_FRAM_1x1_lat].ravel()
Chl_total_interp_FRAM = Chl.Chl_total_interp[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat].ravel()

# TEP
TEP_0_interp_FRAM_mugXeqL = TEP_0_interp_mugXeqL[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat].ravel()

# 2 layer data ######################
# Chl
Chl_n2_total_interp_FRAM = Chl_n2.Chl_total_interp[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat].ravel()

# TEP
TEP_n2_mean = (TEP_0_interp_mugXeqL + TEP_1_interp_mugXeqL)/2
TEP_n2_interp_FRAM_mugXeqL = TEP_n2_mean[mask_FRAM_1x1_lon][:,mask_FRAM_1x1_lat].ravel()

# plotting relation ######################
x1, y1 = np.log10(set_Chl_min(0.01,Chl_total_interp_FRAM,TEP_0_interp_FRAM_mugXeqL))

x2, y2 = np.log10(set_Chl_min(0.05,Chl_total_interp_FRAM,TEP_0_interp_FRAM_mugXeqL))

# 2 layers ######################
x1_n2, y1_n2 = np.log10(set_Chl_min(0.01,Chl_n2_total_interp_FRAM,TEP_n2_interp_FRAM_mugXeqL))
x2_n2, y2_n2 = np.log10(set_Chl_min(0.05,Chl_n2_total_interp_FRAM,TEP_n2_interp_FRAM_mugXeqL))

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.plot(x1,y1,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.01 mg m$^{-3}$\nFram Strait')
plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x1,y1)
plt.plot(x1, res.intercept + res.slope*x1, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.subplot(2,2,2)
plt.plot(x2,y2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.1 mg m$^{-3}$\nFram Strait')
#plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x2,y2)
plt.plot(x2, res.intercept + res.slope*x2, 'r', label='fitted line')
plt.text(-2,0.5,log)

# 2 layer data ################
plt.subplot(2,2,3)
plt.plot(x1_n2,y1_n2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.01 mg m$^{-3}$\nn_layer = 2\nFram Strait')
plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x1_n2,y1_n2)
plt.plot(x1, res.intercept + res.slope*x1, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.subplot(2,2,4)
plt.plot(x2_n2,y2_n2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.1 mg m$^{-3}$\nn_layer = 2\nFram Strait')
#plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x2_n2,y2_n2)
plt.plot(x2, res.intercept + res.slope*x2, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.tight_layout()
if savefig:
    plt.savefig(savepath+'Chl_TEP_FRAM_regression.png', dpi=300)



# Seasonal cycle

### calculating

In [ ]:
### calculating mean TEP per month 
months = np.arange(1,13,1)
depth_list = (0,5,10,20)
verbose = True

In [ ]:


# define storage containers for mean data in shape (months, depths)
TEP_glob_total_m = np.zeros((len(months),len(depth_list)))
TEP_ao_total_m = np.zeros((len(months),len(depth_list)))
TEP_FRAM_total_m = np.zeros((len(months),len(depth_list)))
TEP_so_total_m = np.zeros((len(months),len(depth_list)))

TEP_glob_aux = np.zeros((len(years_last10),len(months)))
TEP_so_aux = np.zeros((len(years_last10),len(months)))
TEP_FRAM_aux = np.zeros((len(years_last10),len(months)))
TEP_ao_aux = np.zeros((len(years_last10),len(months)))

for idepth in range(len(depth_list)):
    if verbose:
        print('Processing depth = {} m'.format(depth_list[idepth]))
    
    # load data for one depth and calculate monthly mean
    for i in range(len(years_last10)):
        ind_year = int(years_last10[i])
        
        if verbose:
            print('loading {0}'.format(ind_year))

        TEP_raw = pf.get_data(resultpath, "TEP", ind_year, mesh, how="ori", 
                            depth=int(depth_list[idepth]), compute=False,  silent=True)#np.invert(verbose))
        # TEP:units = "mmolC/m3"
        
        TEP_glob_aux[i,:] = np.nanmean(TEP_raw,axis=1)
        TEP_ao_aux[i,:] = np.nanmean(TEP_raw[:,mask_AO],axis=1)
        TEP_FRAM_aux[i,:] = np.nanmean(TEP_raw[:,mask_FRAM],axis=1)
        TEP_so_aux[i,:] = np.nanmean(TEP_raw[:,mask_SO],axis=1)
        
    #TEP_mmean = np.nanmean(TEP_glob_aux,axis=0)
    TEP_glob_total_m[:,idepth] = np.nanmean(TEP_glob_aux,axis=0)
    TEP_ao_total_m[:,idepth] = np.nanmean(TEP_ao_aux,axis=0)
    TEP_FRAM_total_m[:,idepth] = np.nanmean(TEP_FRAM_aux,axis=0)
    TEP_so_total_m[:,idepth] = np.nanmean(TEP_so_aux,axis=0)

### plotting

In [ ]:
# plotting total TEP -------------------------------------------------------------------------------        
fig = plt.figure(figsize=(10,7), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (0,15.5)

plt.subplot(2, 2, 1)
plt.plot(months,TEP_glob_total_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_glob_total_m[:,1],'.-',label='5-10 m')
plt.plot(months,TEP_glob_total_m[:,2],'.-',label='10-20 m')
plt.plot(months,TEP_glob_total_m[:,3],'.-',label='20-30 m')
plt.title('Global mean TEP ({0}-{1})'.format(years_last10[0],years_last10[-1]))
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 2)
plt.plot(months,TEP_ao_total_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_ao_total_m[:,1],'.-',label='5-10 m')
plt.plot(months,TEP_ao_total_m[:,2],'.-',label='10-20 m')
plt.plot(months,TEP_ao_total_m[:,3],'.-',label='20-30 m')
plt.title('North of 70 N: mean monthly TEP')
#plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 3)
plt.plot(months,TEP_so_total_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_so_total_m[:,1],'o',label='5-10 m')
plt.plot(months,TEP_so_total_m[:,2],'*',label='10-20 m')
plt.plot(months,TEP_so_total_m[:,3],'*',label='20-30 m')
plt.title('S of -50 mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 2, 4)
plt.plot(months,TEP_FRAM_total_m[:,0],'.-',label='0-5 m')
plt.plot(months,TEP_FRAM_total_m[:,1],'.-',label='5-10 m')
plt.plot(months,TEP_FRAM_total_m[:,2],'.-',label='10-20 m')
plt.plot(months,TEP_FRAM_total_m[:,3],'.-',label='20-30 m')
plt.title('Fram Strait mean TEP')
#plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')


if savefig:
    plt.savefig(savepath+'TEP_monthly_multipledepths.png',dpi=300)

In [ ]:
# plotting Arctic TEP [mug /L]    

conv = 12.01 # [mmol C/m3] -> [mug /L]  

fig = plt.figure(figsize=(5,4), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (0,190)

plt.plot(months,TEP_FRAM_total_m[:,0]*conv,'.-',label='0-5 m')
plt.plot(months,TEP_FRAM_total_m[:,1]*conv,'.-',label='5-10 m')
plt.plot(months,TEP_FRAM_total_m[:,2]*conv,'.-',label='10-20 m')
plt.plot(months,TEP_FRAM_total_m[:,3]*conv,'.-',label='20-30 m')

# adding observation data
plt.axvline(x=6, linewidth=.70, linestyle='--', color='black')
plt.axvline(x=7, linewidth=.70, linestyle='--', color='black')
plt.axvline(x=9, linewidth=.70, linestyle='--', color='black')
plt.scatter(6,8.4, marker='s', color='k', label='E2020')
plt.scatter((7,9),(21,7), marker='D', color='k', label='J2020')

plt.xlabel('Month\n\nMonthly TEP conc. in Fram Strait\nMean of {0}-{1}'.format(
    years_last10[0],years_last10[-1]),fontsize=13)
plt.ylabel(r'TEP [$\mu$g C L$^{-1}$]',fontsize=13)
plt.ylim(ylim)
plt.legend(loc='upper left',fontsize=13)

if True:
    plt.savefig(savepath+'TEP_monthly_multipledepths_FRAM_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]),dpi=300)

In [ ]:
### differences in Southern Ocean
fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (-0.01,0.01)

plt.subplot(2, 3, 3)
plt.plot(months,TEP_so_total_m[:,0]-TEP_so_total_m[:,0],'.-',label='0 m')
plt.plot(months,TEP_so_total_m[:,1]-TEP_so_total_m[:,0],'o-',label='5 m')
plt.plot(months,TEP_so_total_m[:,2]-TEP_so_total_m[:,0],'*-',label='10 m')
plt.plot(months,(TEP_so_total_m[:,3]-TEP_so_total_m[:,0])/50,'*-',label='0.02*(20 m)')
plt.title('S of -50 mean TEP')
plt.xlabel('months\nSurface layer - layer at certain depth')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')



# Monthly map

In [ ]:
def get_monthly_data(var_name, years=years_last10, verbose=True): 
    TEP_monthly_aux = np.zeros((len(years),len(months),len(mesh.x2),len(mesh.zlev)-1))

    if verbose:
        print('Processing all depths vor variable "{}"'.format(var_name))

    # load data for one depth and calculate monthly mean
    for i in range(len(years)):
        ind_year = int(years[i])

        if verbose:
            print('loading {0}'.format(ind_year))

        TEP_raw = pf.get_data(resultpath, var_name, ind_year, mesh, how="ori", 
                            #depth=depth, 
                            compute=False,  silent=True)#np.invert(verbose))
        # TEP:units = "mmolC/m3"
        #print(np.shape(TEP_raw))

        for m in range(len(months)):
            TEP_monthly_aux[i,m,:,:] = TEP_raw[m,:,:]

    if verbose:
        print('Processed {0} years of input data\nResulting shape: {1}'.format(
            len(years_last10),np.shape(TEP_monthly_aux)))

    # monthly average over years
    var_monthly_alldepths = np.nanmean(TEP_monthly_aux,axis=0)
    print('\nMonthly mean over years \nResulting shape {0}'.format(np.shape(var_monthly_alldepths)))

    return var_monthly_alldepths

In [ ]:
# get TEP data
TEP_monthly_alldepths = get_monthly_data('TEP', years=years_last10, verbose=True)

### interpolating and converting

In [ ]:
# interpolating and converting Arctic TEP [mug /L]    

conv = 12.01 # [mmol C/m3] -> [mug /L]  

# surface layer
TEP_monthly_0 = TEP_monthly_alldepths[:,:,0]

# July
TEP_monthly_0_interp_JUL = pf.fesom2regular(data = TEP_monthly_0[6,:],mesh = mesh,lons = londic,lats = latdic)
TEP_monthly_0_interp_JUL = TEP_monthly_0_interp_JUL * conv

# September
TEP_monthly_0_interp_SEP = pf.fesom2regular(data = TEP_monthly_0[8,:],mesh = mesh,lons = londic,lats = latdic)
TEP_monthly_0_interp_SEP = TEP_monthly_0_interp_SEP * conv


### plotting

In [ ]:
from shapely.geometry.polygon import LinearRing

ring1 = LinearRing(list(zip([-20,-20,10,10], [77,82,82,77])))

#box_FRAM =[-20, 10, 77, 82] ## see above


In [ ]:
# North Polar Stereo Projection
fig = plt.figure(figsize=(15,10))
plot_extent = [-180, 180, 70, 90]

ax1 = plt.subplot(1,2,1,projection = ccrs.NorthPolarStereo())
m1 = ax1.pcolormesh(londic, latdic, TEP_monthly_0_interp_JUL, 
    transform = ccrs.PlateCarree(),
    #norm=colors.LogNorm(vmin=np.nanmin(TEP_0_interp),vmax=np.nanmax(TEP_0_interp)),
    cmap='viridis')
mygrid(ax1,grid=True)
ax1.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax1.set_title('July',fontsize=20)
ax1.add_geometries([ring1], ccrs.PlateCarree(), facecolor='None', alpha=1, edgecolor='red',linewidth = 3)

ax2 = plt.subplot(1,2,2,projection = ccrs.NorthPolarStereo())
m2 = ax2.pcolormesh(londic, latdic, TEP_monthly_0_interp_SEP, 
    transform = ccrs.PlateCarree(),
    #norm=colors.LogNorm(vmin=np.nanmin(TEP_0_interp),vmax=np.nanmax(TEP_0_interp)),
    cmap='viridis')
mygrid(ax2,grid=True)
ax2.set_extent(plot_extent, crs=ccrs.PlateCarree())
ax2.set_title('September',fontsize=20)
ax2.add_geometries([ring1], ccrs.PlateCarree(), facecolor='None', alpha=1, edgecolor='red',linewidth = 3)


# colorbar
cbar = fig.colorbar(m1, ax=[ax1,ax2],orientation = 'horizontal',
                    fraction=0.1, pad=0.05, shrink = 0.8) 
cbar.set_label('[$\mu$g C L$^{{-1}}$]\nMonthly mean TEP conc. at 0-5 m ({0}-{1})'.format(
    years_last10[0],years_last10[-1]), 
               fontsize=20)

plt.tight_layout()

if True:
    plt.savefig(savepath+'TEP_interp_NPS_JUL_SEP_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]), dpi=300)

# Fram Strait Transect

In [ ]:
#box_FRAM =[-20, 10, 77, 82] ## see above

lon_start = 0
lon_end = 5
lat_start = 75
lat_end = 83
npoints   = 50
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

if True:
    pf.plot_transect_map(lonlat, mesh)

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)

In [ ]:
# get data
TEPall = pf.get_data(resultpath, "TEP", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(TEPall))

print('TEP loaded for years {0}-{1}\nmin = {3:4.4f}, max = {4:4.4f}, mean = {5:4.4f} [mmol/m3]'.format(
    last_year-10,last_year,depth, np.nanmin(TEPall), np.nanmax(TEPall), np.nanmean(TEPall)))

dist_label= 'Fram Strait \nTransect at lat {0}-{1}, lon {2}-{3}'.format(lat_start,lat_end,lon_start,lon_end)
print(dist_label)
#pf.plot_transect_map(lonlat, mesh)

# extract transect data
dist, tfTEP = pf.get_transect(TEPall,mesh,lonlat)


In [ ]:
# get PCHO data
PCHO = pf.get_data(resultpath, "PCHO", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(TEPall))

print('PCHO loaded for years {0}-{1}\nmin = {3:4.4f}, max = {4:4.4f}, mean = {5:4.4f} [mmol/m3]'.format(
    last_year-10,last_year,depth, np.nanmin(PCHO), np.nanmax(PCHO), np.nanmean(PCHO)))

dist_label= 'Fram Strait \nTransect at lat {0}-{1}, lon {2}-{3}'.format(lat_start,lat_end,lon_start,lon_end)
print(dist_label)
#pf.plot_transect_map(lonlat, mesh)

# extract transect data
dist, tfPCHO = pf.get_transect(PCHO,mesh,lonlat)



In [ ]:
# get CHLOROPHYLL data
PhyChl_Fram = pf.get_data(resultpath, "PhyChl", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)
DiaChl_Fram = pf.get_data(resultpath, "PhyChl", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

Chl_SUM_Fram = PhyChl_Fram + DiaChl_Fram

print('Chlorophyll data loaded for years {0}-{1}\nmin = {3:4.4f}, max = {4:4.4f}, mean = {5:4.4f} [mg/m3]'.format(
    last_year-10,last_year,depth, np.nanmin(Chl_SUM_Fram), np.nanmax(Chl_SUM_Fram), np.nanmean(Chl_SUM_Fram)))

dist_label= 'Fram Strait \nTransect at lat {0}-{1}, lon {2}-{3}'.format(lat_start,lat_end,lon_start,lon_end)
print(dist_label)
#pf.plot_transect_map(lonlat, mesh)

# extract transect data
dist, tfChl = pf.get_transect(Chl_SUM_Fram,mesh,lonlat)

#### plotting all together

In [ ]:
time_label = '\nMean over {0}-{1}'.format(last_year-10,last_year)

#pf.plot_xyz?
pf.plot_xyz(mesh, 
            [tfChl,tfPCHO,tfTEP],
            dist,
            maxdepth=100,
            ncols=3,
            xlabel='Distance\n',
            title=['Chl.a','Fram Strait Transect\nPCHO','TEP'],
            label=['Chl.a [mg/m3]'+time_label,'PCHO-C [mmol/m3]'+time_label,'TEP-C [mmol/m3]'+time_label],
            #levels=np.linspace(0,5,10),
            figsize=(15,6),
            cmap='viridis')


if True:
    plt.savefig(savepath+'TEP_Fram_Transect_{0}_{1}.png'.format(
    years_last10[0],years_last10[-1]), dpi=300)

# Atlantic Transect

In [ ]:
lon_start = -60
lat_start = -80
lon_end = 10
lat_end = 83
npoints   = 100
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

if True:
    pf.plot_transect_map(lonlat, mesh)

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)

In [ ]:
# get data
TEPall = pf.get_data(resultpath, "TEP", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(TEPall))

print('TEP loaded for years {0}-{1}\nmin = {3}, max = {4}, mean = {5} [mmol/m3]'.format(
    last_year-10,last_year,depth, np.nanmin(TEPall), np.nanmax(TEPall), np.nanmean(TEPall)))

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)

# extract transect data
dist, tfTEP = pf.get_transect(TEPall,mesh,lonlat)

pf.plot_xyz(mesh, 
            tfTEP,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='TEP-C [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis')

# PCHO

In [ ]:
units = 'org C [mmol m$^{-3}$]'
label='Polysaccharides {0}-{1} at {2} m'.format(last_year-10,last_year,depth)

PCHO = pf.get_data(resultpath, "PCHO", years=years_last10, 
                   mesh = mesh, runid=runid, how="mean", compute=True, depth=depth)

fig = pf.plot(mesh, PCHO, mapproj = mapproj, units=units, titles=label)

In [ ]:
# Atlantic transect 

# load data
PCHOall = pf.get_data(resultpath, "PCHO", years=years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(PCHO))

print('PCHO loaded for years {0}-{1}\nmin = {3}, max = {4}, mean = {5} [mmol/m3]'.format(
    years_last10[0],years_last10[-1],depth, np.nanmin(PCHOall), np.nanmax(PCHOall), np.nanmean(PCHOall)))

# extract transect data
dist, tfPCHO = pf.get_transect(PCHOall,mesh,lonlat)

pf.plot_xyz(mesh, 
            tfPCHO,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='PCHO [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis',
            figsize=(15, 6))

# Intracellular Carbon

In [ ]:
# Diatom carbon
depth = 0
NCfesom = resultpath + '/DiaC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Diatom intracell. Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'intracellular C [mmol C m$^{-3}$]'

DiaC = pf.get_data(resultpath, "DiaC", years=years_last10,
                   mesh=mesh, runid=runid, how="mean", depth=depth)
fig = pf.plot(mesh, DiaC, mapproj = mapproj, units=units, titles=label)

In [ ]:
# Phytplankton carbon

NCfesom = resultpath + '/PhyC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Phytoplankton intracell. Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'intracellular C [mmol C m$^{-3}$]'

PhyC = pf.get_data(resultpath, "PhyC", years=years_last10,
                   mesh=mesh, runid=runid, how="mean", depth = depth)

fig = pf.plot(mesh, PhyC, mapproj = mapproj, units=units, titles=label)

# Detritus Carbon

In [ ]:
NCfesom = resultpath + '/DetC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='DetC {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'Detritus C [mmol m$^{-3}$]'

DetC = pf.get_data(resultpath, "DetC", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DetC, mapproj = mapproj, units=units, titles=label)

# extracellular carbon

In [ ]:
NCfesom = resultpath + '/EOC.fesom.1948.nc'
!ncdump -h $NCfesom

#label='TEP {0}-{1} at {2} m'.format(years_last10[0],years_last10[-1],depth)
#units = 'TEP-C [mmol m$^{-3}$]'
#TEP = pf.get_data(resultpath, "EOC", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depth)
#fig = pf.plot(mesh, TEP, mapproj = mapproj, units=units, titles=label)

# NPP

In [ ]:
# map of total Diatom NPP 

NCfesom = resultpath + '/NPPd.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mmolC/m2/d to mg C/m2/d

label='Diatom NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPd = pf.get_data(resultpath, "NPPd", 
                   years_last10,
                   mesh, runid=runid, how="mean", compute=True, silent=True)

#print(np.shape(NPPd))
# NPP is already vertically integrated!

NPPd = NPPd * 12.01
fig = pf.plot(mesh, NPPd, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
# map of nanophytoplankton NPP

NCfesom = resultpath + '/NPPn.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mg C/m2/d

label='Nanophytoplankton NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPn = pf.get_data(resultpath, "NPPn", 
                   years_last10,
                   mesh, runid=runid, how="mean", compute=True,silent=True)

#print(np.shape(NPPn))
# NPP is already vertically integrated!

NPPn = NPPn *12.01

fig = pf.plot(mesh, NPPn, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
# total NPP
label='Total Net Primary Production {0}-{1}'.format(last_year-10,last_year)
units = 'Mean total NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPtot = NPPd + NPPn
fig = pf.plot(mesh, NPPtot, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

# DIC

In [ ]:
# DIC
depth = 0
NCfesom = resultpath + '/DIC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Dissolved inorganic Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'DIC [mmol  m$^{-3}$]'
DIC = pf.get_data(resultpath, "DIC", years_last10,mesh, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DIC, mapproj = mapproj, units=units, titles=label)


# export to html

In [ ]:
stop

In [ ]:
!jupyter nbconvert TEP_detailed_analysis.ipynb --output-dir=$htmlpath --output $htmlname --to html --TemplateExporter.exclude_input=True


# Backup

In [ ]:
### old/backup
TEP_monthly_aux = np.zeros((len(years_last10),len(months),len(mesh.x2),len(mesh.zlev)-1))

var_name = 'TEP'
verbose = True

if verbose:
    print('Processing all depths vor variable "{}"'.format(var_name))

# load data for one depth and calculate monthly mean
for i in range(len(years_last10)):
    ind_year = int(years_last10[i])

    if verbose:
        print('loading {0}'.format(ind_year))

    TEP_raw = pf.get_data(resultpath, var_name, ind_year, mesh, how="ori", 
                        #depth=depth, 
                        compute=False,  silent=True)#np.invert(verbose))
    # TEP:units = "mmolC/m3"
    #print(np.shape(TEP_raw))

    for m in range(len(months)):
        TEP_monthly_aux[i,m,:,:] = TEP_raw[m,:,:]

if verbose:
    print('Processed {0} years of input data\nResulting shape: {1}'.format(
        len(years_last10),np.shape(TEP_monthly_aux)))
    
# monthly average over years
TEP_monthly_alldepths = np.nanmean(TEP_monthly_aux,axis=0)
print('Monthly mean over years \nResulting shape {0}'.format(np.shape(TEP_monthly_alldepths)))

